Paper Based I3D


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import cv2
import numpy as np
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import json

# Constants for resizing and sampling
NUM_FRAMES = 64  # Number of frames to sample
FRAME_HEIGHT = 224  # Resize height for I3D
FRAME_WIDTH = 224  # Resize width for I3D
CHANNELS = 3  # RGB channels
VIDEO_PATH = "./Dataset/videos"
SAVE_PATH = "./Dataset/I3D"
METADATA_PATH = "DataSet/WLASL_v0.3.json"



In [3]:
# Create SAVE_PATH directory if it doesn't exist
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)
    print(f"Created SAVE_PATH directory: {SAVE_PATH}")
else:
    print(f"SAVE_PATH already exists: {SAVE_PATH}")

# Load metadata from the JSON file
with open(METADATA_PATH, 'r') as f:
    metadata = json.load(f)

video_label_map = {}
for entry in metadata:
    label = entry['gloss']  # Action label
    for instance in entry['instances']:
        video_id = int(instance['video_id'])
        video_label_map[video_id] = label  # Map video ID to label

Created SAVE_PATH directory: /content/drive/MyDrive/Colab Notebooks/AAI-521/Final Project/I3D_processed


In [4]:
# Function to preprocess video for I3D
def preprocess_video(video_path, num_frames=NUM_FRAMES, target_size=(FRAME_HEIGHT, FRAME_WIDTH)):
    """Preprocess a video file for I3D input"""
    try:
        cap = cv2.VideoCapture(video_path)
        frames = []
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # Total number of frames in the video

        if frame_count == 0:
            print(f"Skipping video (no frames found): {video_path}")
            return None

        # Sample frames uniformly from the video
        for i in range(min(num_frames, frame_count)):
            cap.set(cv2.CAP_PROP_POS_FRAMES, int(i * frame_count / num_frames))  # Set the frame position
            success, frame = cap.read()

            if success:
                frame = cv2.resize(frame, target_size)  # Resize frame to target size
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert to RGB
                frame = frame.astype(np.float32) / 255.0  # Normalize pixel values (0-1)
                frames.append(frame)

        cap.release()

        frames = np.stack(frames, axis=0)  # Stack frames to create a (num_frames, height, width, channels) tensor

        # If fewer frames are available, pad with zeros (optional)
        if len(frames) < num_frames:
            frames = np.pad(frames, ((0, num_frames - len(frames)), (0, 0), (0, 0), (0, 0)), 'constant')

        return frames
    except Exception as e:
        print(f"Error processing video {video_path}: {e}")
        return None


In [5]:
# Process each video file sequentially and save it immediately to free up memory
def process_and_save_video(video_file):
    """Process a single video file and save its frames."""
    video_path = os.path.join(VIDEO_PATH, video_file)

    # Extract the video ID from the file name (assuming video file names are in the format of video_id.mp4)
    video_id = int(video_file.split('.')[0])

    # Get the label (action) from video_label_map
    if video_id in video_label_map:
        label = video_label_map[video_id]
    else:
        print(f"Warning: No label found for video {video_file}")
        return None

    # Preprocess the video
    frames = preprocess_video(video_path)

    if frames is not None:
        # Save the processed frames as a .npy file
        action_save_dir = os.path.join(SAVE_PATH, label)
        os.makedirs(action_save_dir, exist_ok=True)

        # Save processed frames
        np.save(os.path.join(action_save_dir, video_file.replace('.mp4', '_frames.npy')), frames)
        print(f"Saved {video_file} as {label}_frames.npy")
        return True

    return None

In [6]:
# Get list of all mp4 files in VIDEO_PATH
video_files = [file for file in os.listdir(VIDEO_PATH) if file.endswith('.mp4')]

In [ ]:
# Process each video sequentially
processed_videos = 0
for video_file in tqdm(video_files, desc="Processing Videos", unit="video"):
    if process_and_save_video(video_file):
        processed_videos += 1

print(f"Processed and saved frames for {processed_videos} videos in {SAVE_PATH}.")

Processing Videos:   0%|          | 1/11980 [00:01<3:34:11,  1.07s/video]

Saved 65449.mp4 as decide_frames.npy


Processing Videos:   0%|          | 2/11980 [00:02<4:19:03,  1.30s/video]

Saved 65394.mp4 as conflict_frames.npy


Processing Videos:   0%|          | 3/11980 [00:03<3:28:52,  1.05s/video]

...

Saved 65294.mp4 as can_frames.npy


Processing Videos:  16%|█▌        | 1905/11980 [51:58<3:31:33,  1.26s/video]

Saved 65279.mp4 as butter_frames.npy


Processing Videos:  16%|█▌        | 1906/11980 [52:00<3:41:37,  1.32s/video]

Saved 65304.mp4 as card_frames.npy


In [11]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Load InceptionV3 pretrained on ImageNet
def load_imagenet_pretrained(input_shape=(90, 224, 224, 3)):
    base_model = tf.keras.applications.InceptionV3(
        include_top=False,  # Exclude the top classification layer
        weights='imagenet',  # Pretrained on ImageNet
        input_shape=input_shape[1:]  # Spatial dimensions (no temporal yet)
    )
    return base_model


In [15]:
def inflate_2d_to_3d(base_model):
    # Wrap base_model with TimeDistributed to handle temporal input
    model = layers.TimeDistributed(base_model, input_shape=(90, 224, 224, 3))

    # Add 3D Convolutional layers for temporal modeling
    model = layers.Conv3D(64, (3, 3, 3), activation='relu', padding='same')(model)
    model = layers.MaxPooling3D(pool_size=(2, 2, 2))(model)

    model = layers.Conv3D(128, (3, 3, 3), activation='relu', padding='same')(model)
    model = layers.MaxPooling3D(pool_size=(2, 2, 2))(model)

    model = layers.Flatten()(model)
    return model



In [17]:
def build_model(num_classes=2000):
    # Input tensor for video sequences (90 frames, 224x224 RGB images)
    input_tensor = layers.Input(shape=(90, 224, 224, 3))

    # Load base model pretrained on ImageNet
    base_model = load_imagenet_pretrained()  # Pretrained InceptionV3

    # Apply TimeDistributed wrapper
    x = layers.TimeDistributed(base_model)(input_tensor)

    # Add 3D convolutions for temporal modeling
    x = layers.Conv3D(64, (3, 3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling3D(pool_size=(2, 2, 2))(x)

    x = layers.Conv3D(128, (3, 3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling3D(pool_size=(2, 2, 2))(x)

    # Flatten the output and add fully connected layers
    x = layers.Flatten()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)

    # Final classification layer for WLASL
    output_tensor = layers.Dense(num_classes, activation='softmax')(x)

    # Create the model
    model = models.Model(inputs=input_tensor, outputs=output_tensor)
    return model



In [21]:
# Build the model
num_classes = 2000  # Number of classes in WLASL
model = build_model(num_classes=num_classes)

# Freeze the base layers
for layer in model.layers[:-2]:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()




Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)           │ (None, 90, 224, 224, 3)     │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_4 (TimeDistributed) │ (None, 90, 5, 5, 2048)      │      21,802,784 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_5 (Conv3D)                    │ (None, 90, 5, 5, 64)        │       3,539,008 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_2 (MaxPooling3D)       │ (None, 45, 2, 2, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_6 (Conv3D)                    │ (None, 45, 2, 2, 128)       │         221,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_3 (MaxPooling3D)       │ (None, 22, 1, 1, 128)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 2816)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │       1,442,304 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2000)                │       1,026,000 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 28,031,408 (106.93 MB)

 Trainable params: 1,026,000 (3.91 MB)

 Non-trainable params: 27,005,408 (103.02 MB)